In [16]:
import os
import pandas as pd
import arcpy
from arcpy import env
from arcpy.management import *
from arcpy.sa import *
from arcpy.da import *
from arcpy.conversion import *
from arcpy.analysis import *

In [12]:
# tool function
# 创建渔网
def create_net(output_file,extent_file,cell_size):
    """参数说明：
        output_file: 输出文件
        extent_file: 输入范围文件
        cell_size: 生成的cell大小
"""
    proj = arcpy.Describe(extent_file).spatialReference
    extent_x_min = arcpy.Describe(extent_file).extent.XMin
    extent_y_min = arcpy.Describe(extent_file).extent.YMin
    extent_x_max = arcpy.Describe(extent_file).extent.XMax
    extent_y_max = arcpy.Describe(extent_file).extent.YMax
    with arcpy.EnvManager(outputCoordinateSystem = proj):
        arcpy.management.CreateFishnet(out_feature_class=output_file,origin_coord=f"{extent_x_min} {extent_y_min}",y_axis_coord=f"{extent_x_min} {extent_y_min+10}",cell_width=cell_size,cell_height=cell_size,number_rows=None,number_columns=None,corner_coord=f"{extent_x_max} {extent_y_max}",labels="NO_LABELS",template="#",geometry_type="POLYGON")
# 面转点
def polygon_point(in_feature, out_feature):
    """参数说明：
        in_feature: 输入面
        out_feature: 输出点
    """
    proj = arcpy.Describe(in_feature).spatialReference
    with arcpy.EnvManager(outputCoordinateSystem=proj):
        arcpy.management.FeatureToPoint(in_features=in_feature,out_feature_class=out_feature,point_location="INSIDE")

# 筛选点

def select_point(input_polygon,input_point,output_point):
    """参数说明：
        input_polygon: 选择面
        input_point: 待选择点
        output_point: 选择点
    """
    polygon_layer = "polygonLayer"
    point_layer = "pointLayer"
    arcpy.MakeFeatureLayer_management(input_polygon, polygon_layer)
    arcpy.MakeFeatureLayer_management(input_point, point_layer)
    arcpy.SelectLayerByLocation_management(in_layer=point_layer, overlap_type="INTERSECT", select_features=polygon_layer,search_distance=0,selection_type='NEW_SELECTION',invert_spatial_relationship="NOT_INVERT")
    # 导出选择的要素
    arcpy.CopyFeatures_management(point_layer, output_point)

# 添加字段
def add_field(input_table,field_name,field_type='TEXT'):
    """参数说明：
        input_table: 输入数据表
        field_name: 字段名
        field_type: 字段类型"""
    arcpy.AddField_management(input_table,field_name,field_type)

# 空间连接
def perform_spatial_join(target_layer_path, join_layer_path, output_layer_path, field_mapping_dict):
    """参数说明：
        target_layer_path: 目标图层路径
        join_layer_path: 连接图层路径
        output_layer_path: 输出图层路径
        field_mapping_dict: 字段映射字典
    """
    # 创建 FieldMappings 对象
    field_mappings = arcpy.FieldMappings()

    # 为每个字段对创建 FieldMap 并添加到 FieldMappings
    for target_field, join_field in field_mapping_dict.items():
        field_map = arcpy.FieldMap()
        
        # 添加目标字段和连接字段到FieldMap
        field_map.addInputField(target_layer_path, target_field)
        field_map.addInputField(join_layer_path, join_field)
        
        # 设置输出字段属性
        output_field = field_map.outputField
        output_field.name = target_field
        output_field.aliasName = target_field
        field_map.outputField = output_field

        # 将 FieldMap 添加到 FieldMappings
        field_mappings.addFieldMap(field_map)

    # 执行空间连接操作
    arcpy.analysis.SpatialJoin(target_layer_path, join_layer_path, output_layer_path,"JOIN_ONE_TO_ONE", "KEEP_ALL", field_mappings)


# 点采样
def sample_point(point_,raster_,out_name):
    """根据栅格采样点,输出为表格"""
    Sample(raster_,point_,out_name,"NEAREST", "OBJECTID", "CURRENT_SLICE", None, '', None, None, "ROW_WISE", "TABLE")
    return None

# 导出CSV
def export_csv(table_,out_path,out_name):
    """参数说明：
        table_: 输入数据表
        out_path: 输出路径
        out_name: 输出表名"""
    TableToTable(table_,out_path,out_name)
    return None


In [3]:
# 设置工作空间
env.workspace = r'D:\ArcGISProjects\workspace\duyun\basedata_vector.gdb'


In [13]:
# 创建渔网
create_net('dunyun_grid','dunyun_tl',500)

In [15]:
# 要素转点
polygon_point('dunyun_grid','dunyun_inside_point')

In [17]:
# 按位置选择
select_point('dunyun_tl','dunyun_inside_point','select_point')

In [4]:
# 添加字段
input_features = 'select_point'
field_list = ['tl','yl','ts','tz']
for one_field in field_list:
    # 添加字段
    add_field(input_table=input_features,field_name=one_field,field_type='TEXT')

In [15]:
# 空间连接赋予地类属性
# 定义目标要素图层和连接要素图层的路径
targetFeatures = "select_point"
joinFeatures = "dunyun_tl"
outputFeatures = "join_point"
fields_mapping = {
    "TL": "土类",
    "YL": "亚类",
    "TS": "土属",
    "TZ": "土种"
}
# 空间连接
perform_spatial_join(targetFeatures,joinFeatures,outputFeatures,fields_mapping)

In [24]:
# 对特征因子采样

# 用于采样的特征因子列表
feature_list = ['BIO', 'PRE', 'SRA', 'TMP', 'VAP', 'WIN', 'NDVI', 'TDQS', 'LIGHT', 'AnalyticalHillshading', 'Aspect', 'ChannelNetworkBaseLevel', 'ChannelNetworkDistance', 'ClosedDepressions', 'ConvergenceIndex', 'LSFactor', 'PlanCurvature', 'ProfileCurvature', 'RelativeSlopePosition', 'Slope', 'TopographicWetnessIndex', 'TotalCatchmentArea', 'ValleyDepth', 'DEM', 'LON', 'LAT', 'prediction_ph']

# 选择用于采样的特征因子数据库
stander_raster_gdb = r"D:\ArcGISProjects\workspace\duyun\featuressdata_dy.gdb"
env.workspace = stander_raster_gdb # 切换工作空间用于采样
# 选择用于采样的要素类
point_data = r"D:\ArcGISProjects\workspace\duyun\basedata_vector.gdb\join_point"

# 用于存放采样表的数据库路径
sample_gdb_path = r"D:\ArcGISProjects\workspace\duyun\tabledata.gdb"


# 使用Delete_management函数删除数据库中的所有内容
try:
    arcpy.Delete_management(sample_gdb_path)
except:
    pass
# 再创建一个新的数据库
arcpy.management.CreateFileGDB(os.path.dirname(sample_gdb_path), "tabledata", "CURRENT")
# 逐个采样并保存到csv文件
for one_feature in feature_list:
    sample_point(point_data,one_feature,os.path.join(sample_gdb_path,one_feature))

ExecuteError: ERROR 000258: 输出 D:\ArcGISProjects\workspace\tabledata.gdb 已存在
执行(CreateFileGDB)失败。


In [19]:
# 导出CSV表用于分析
env.workspace = os.path.join(sample_gdb_path) # 切换工作空间用于导出csv文件
# 采样点数据名称
sample_name = 'duyun_point'
filed_list = [_.name for _ in arcpy.ListFields(point_data)]
print(filed_list)
# 需要保留的字段
elements_yes = ['OBJECTID', 'TL','YL','TS','TZ']
filter_list = [_ for _ in filed_list if _ in elements_yes]
print(filter_list)


['OBJECTID', 'Shape', 'Join_Count', 'TARGET_FID', 'TL', 'YL', 'TS', 'TZ']
['OBJECTID', 'TL', 'YL', 'TS', 'TZ']


In [ ]:
# 读取数据表并保存到csv文件
result_df = pd.DataFrame(arcpy.da.FeatureClassToNumPyArray(point_data,filter_list))
result_df.rename(columns={"OBJECTID":sample_name},inplace=True)
#  读取每个表的最后一个字段的数据,存储每个表的最后一个字段的数据
for table in feature_list:
    # 将表转换为pandas数据帧
    df = pd.DataFrame(arcpy.da.TableToNumPyArray(table, "*"))  # 确保数据表中无空值
    # 提取最后一个字段的数据
    merged_df = df[[sample_name, df.columns[-1]]]
    # 合并
    result_df = pd.merge(result_df, merged_df, on=[sample_name])
# 保存到csv文件
result_df.rename(columns=dict(zip(result_df.columns[-len(feature_list):], feature_list)),inplace=True)
result_df.drop(result_df.columns[0],axis=1,inplace=True)
# result_df.rename(columns={"PH":"PH_T"},inplace=True)
result_df.to_csv(os.path.join(sample_csv,sample_csv_name),index=False)